In [29]:
import numpy as np
import pandas as pandas
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joaju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Read in the whole story

In [30]:
story_path = "/Users/joaju/OneDrive/LanguageModel/SherlockZip/sherlock/sherlock/"

# Reads in all of the textfile from the path and stores it in a list;

def read_story(story_path):
    txt = []
    for _, _, files in os.walk(story_path):
        for file in files:
            with open(story_path+file) as f:
                for line in f:
                    line = line.strip()
                    if line != '':
                        txt.append(line)

    return txt

story = read_story(story_path)
print("Number of lines : ", len(story))

Number of lines :  215663


# Filter non-english alphabetic words

In [31]:
# Remove non alphabet characters from the txt since we are only interested
# in feeding english words.
# Tokenizing : Converting words in to each state is tokenizing.

def clean_txt(txt):
    cleaned_txt = []
    for line in txt:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_txt += words
    return cleaned_txt

cleaned_story = clean_txt(story)
print("number of words = ", len(cleaned_story))


number of words =  2337745


# Build the Markov Model

Markov model is essentially a graph model, that has many different vertices
which we call each a "state". Each state in this graph represents a single pair of
words and the edges that connect these states have weights equivalent to how many times they appear adjacently in the story.

In [32]:
def markov_model(cleaned_story, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_story)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += cleaned_story[i+j] + " " # current state is a set of n_gram words
            next_state += cleaned_story[i+j+n_gram] + " " # set the next state
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model: # include this word in dictionary if not existent
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1 # Start counting the number of appearance
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1 # Increment the number of times next_state came after current state
            else:
                markov_model[curr_state][next_state] = 1

        
    # Calculating transition probabilities
    # By dividing the counted values by total transition values.
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count / total

    return markov_model
        



In [33]:
markov_chain = markov_model(cleaned_story)

In [34]:
print("number of states = ", len(markov_chain.keys()))

number of states =  208801


In [35]:
print("All possible transitions from the state 'the game' : \n")
print(markov_chain['the game'])

All possible transitions from the state 'the game' : 

{'your letter': 0.02702702702702703, 'was up': 0.09009009009009009, 'is afoot': 0.036036036036036036, 'for the': 0.036036036036036036, 'was in': 0.02702702702702703, 'is hardly': 0.02702702702702703, 'would have': 0.036036036036036036, 'is up': 0.06306306306306306, 'is and': 0.036036036036036036, 'in their': 0.036036036036036036, 'was whist': 0.036036036036036036, 'in that': 0.036036036036036036, 'the lack': 0.036036036036036036, 'for all': 0.06306306306306306, 'may wander': 0.02702702702702703, 'now a': 0.02702702702702703, 'my own': 0.02702702702702703, 'at any': 0.02702702702702703, 'mr holmes': 0.02702702702702703, 'ay whats': 0.02702702702702703, 'my friend': 0.02702702702702703, 'fairly by': 0.02702702702702703, 'is not': 0.02702702702702703, 'was not': 0.02702702702702703, 'was afoot': 0.036036036036036036, 'worth it': 0.02702702702702703, 'you are': 0.02702702702702703, 'i am': 0.02702702702702703, 'now count': 0.0270270270

# Generate Random Sherlock holmes story

In [63]:
def generate_story(markov_chain, start, limit = 100):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story += curr_state + " "
    while n < limit:
        next_state = random.choices(list(markov_chain[curr_state].keys()),
                                    list(markov_chain[curr_state].values()))
        curr_state = next_state[0]
        story += curr_state + " "
        n += 1
    return story


In [71]:
start = 'dear holmes'
for i in range(21)[1:]:
    print(str(i), ".", generate_story(markov_chain, start, limit=8))
    start = random.choices(list(markov_chain[start].keys()))[0]
    


1 . dear holmes he has some very sharp instrument it was some distance up the small winding which led 
2 . i fear your servant miss morstan he kept his hound to treat them you may be from him 
3 . this means that there is a mr john douglas at all but one thing while they marched in 
4 . that there is pain for you my man however it chanced that even when the moriarty gang was 
5 . on that particular night seven years and the child likewise let it proceed by all means so long 
6 . side a ridge of rocks ended in a very abstruse and complicated one but in any court i 
7 . blow was delivered with such intense earnestness that she could easily do in the hands of the enemy 
8 . struck and two and thirty pounds we shall reach some definite result let the public into my brother 
9 . one and two and i beg of you to ask about father but they were true but could 
10 . that i may place considerable confidence in mr holmes sir i am not a killing its early days 
11 . took the letters in the word lies

The above 20 lines are randomly generated sentences using our markov model.
It has utilized the graph data structure through dictionary to model the probability relationships between
words that have shown up in Sherlock Holmes.